In [116]:
from collections import defaultdict

import numpy as np
import pandas as pd
from tqdm import tqdm

from src.config import c

In [146]:
det_df = pd.read_csv("/app/res/n_nocall_predictions.csv.gz")
det_df = det_df[(det_df["nocall"] == 0) & (det_df["bird"] == 1)]
det_df = (
    det_df[["filename", "end_sec"]]
    .sort_values(["filename", "end_sec"])
    .set_index("filename")
)

In [152]:
detections = {}  # filename -> [[from_s, to_s]...]

In [156]:
with tqdm(total=det_df.shape[0]) as t:

    for ix in det_df.groupby("filename").indices:
        end_secs = det_df.loc[ix]["end_sec"]
        if type(end_secs) != pd.Series:
            end_secs = [end_secs]
        end_secs = list(end_secs)

        detections[ix] = list(map(lambda x: [x - 5, x], end_secs))
        t.update(len(end_secs))

100%|██████████| 2559079/2559079 [00:21<00:00, 121798.65it/s]


In [160]:
for k, vv in detections.items():
    for v in vv:
        assert v[0] >= 0
        assert v[1] >= 0